In [4]:
!pip install germansentiment

import pandas as pd
from germansentiment import SentimentModel

data = pd.read_csv("df_all.csv").sample(1000)
text = list(data.tweet)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt

## Sentiment Sample

### Load Sample

In [5]:
model = SentimentModel()

result = model.predict_sentiment(text)

data["sentiment"] = result

data.to_csv("df_all_sample_with_sentiment.csv")

In [74]:
data_sample = pd.read_csv('df_all_sample_with_sentiment.csv')
data_sample

,Unnamed: 0,Unnamed: 0.1,party,id,conversation_id,created_at,date,user_id,tweet,language,replies_count,retweets_count,likes_count,hashtags,avg_len_of_tweet,sentiment
0,263185,263185,GRUENE,1.429353e+18,1.429349e+18,2021-08-22 10:02:10,2021-08-22 00:00:00,1238824564529995778,@michfleischer @zeitonline @rezomusik @HeikoMa...,de,1.0,0.0,26.0,[],96.0,negative
1,221807,221807,FDP,1.427295e+18,1.426960e+18,2021-08-16 17:42:06,2021-08-16 00:00:00,1425087254393937933,@dan_uschi @mick_noir @fdp @c_lindner Guck dir...,de,2.0,0.0,0.0,[],235.0,negative
2,44335,44335,AFD,1.429667e+18,1.429367e+18,2021-08-23 06:50:05,2021-08-23 00:00:00,1719896598,@Alice_Weidel Es kommen doch keine Afghanen ra...,de,0.0,0.0,0.0,[],288.0,negative
3,114862,114862,CDU,1.429739e+18,1.429699e+18,2021-08-23 11:35:31,2021-08-23 00:00:00,3651150922,@ulfposh @welt @ArminLaschet Der liebe Armin s...,de,0.0,0.0,3.0,[],200.0,neutral
4,28874,28874,AFD,1.427954e+18,1.427859e+18,2021-08-18 13:21:13,2021-08-18 00:00:00,61406432,@InneKomm @AfD_Muenster Eintagsfliegen geben d...,de,0.0,0.0,0.0,[],79.0,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,69965,69965,CDU,1.428828e+18,1.428801e+18,2021-08-20 21:16:05,2021-08-20 00:00:00,1167407058355916800,@mija_roe @SoniaMarima @ArminLaschet @JoachimS...,de,2.0,0.0,0.0,[],155.0,negative
996,249653,249653,GRUENE,1.429886e+18,1.429479e+18,2021-08-23 21:20:41,2021-08-23 00:00:00,3295006025,@JulianFSchmidt @WEEDY42O @Volker_Pudt @warrin...,de,1.0,0.0,0.0,[],277.0,neutral
997,187613,187613,CDU,1.427650e+18,1.427343e+18,2021-08-17 17:14:28,2021-08-17 00:00:00,1309801992315961346,@derya_tn @WerteUnion @ArminLaschet Da weiß ma...,de,0.0,0.0,13.0,"['werteunion', 'afd']",126.0,negative
998,271532,271532,GRUENE,1.428718e+18,1.428354e+18,2021-08-20 15:59:56,2021-08-20 00:00:00,63090014,@KPKuhn @Arishna12 @VeroWendland @oschirg @Die...,de,0.0,0.0,0.0,[],313.0,neutral


### Clean Sample and put Weight on Tweets

In [150]:
def cleaning_sample(csv_path):
    dict_to_numeric = {"negative": -1, "neutral": 0, "positive": 1}
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=['Unnamed: 0',
                                                'Unnamed: 0.1',
                                                "id", 
                                                "conversation_id", 
                                                "user_id", 
                                                "language", 
                                                "hashtags", 
                                                "avg_len_of_tweet",
                                                "replies_count"
                                      ])
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"])
    df_party_name["date"] = pd.to_datetime(df_party_name["date"])
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name["sentiment"].replace(dict_to_numeric, inplace=True)
    df_party_name = df_party_name.reset_index(drop=True)
    return df_party_name

In [151]:
df_sample = cleaning_sample('df_all_sample_with_sentiment.csv')
df_sample

,party,created_at,date,tweet,retweets_count,likes_count,sentiment
0,GRUENE,2021-08-22 10:02:10,2021-08-22,@michfleischer @zeitonline @rezomusik @HeikoMa...,0.0,26.0,-1
1,FDP,2021-08-16 17:42:06,2021-08-16,@dan_uschi @mick_noir @fdp @c_lindner Guck dir...,0.0,0.0,-1
2,AFD,2021-08-23 06:50:05,2021-08-23,@Alice_Weidel Es kommen doch keine Afghanen ra...,0.0,0.0,-1
3,CDU,2021-08-23 11:35:31,2021-08-23,@ulfposh @welt @ArminLaschet Der liebe Armin s...,0.0,3.0,0
4,AFD,2021-08-18 13:21:13,2021-08-18,@InneKomm @AfD_Muenster Eintagsfliegen geben d...,0.0,0.0,0
...,...,...,...,...,...,...,...
995,CDU,2021-08-20 21:16:05,2021-08-20,@mija_roe @SoniaMarima @ArminLaschet @JoachimS...,0.0,0.0,-1
996,GRUENE,2021-08-23 21:20:41,2021-08-23,@JulianFSchmidt @WEEDY42O @Volker_Pudt @warrin...,0.0,0.0,0
997,CDU,2021-08-17 17:14:28,2021-08-17,@derya_tn @WerteUnion @ArminLaschet Da weiß ma...,0.0,13.0,-1
998,GRUENE,2021-08-20 15:59:56,2021-08-20,@KPKuhn @Arishna12 @VeroWendland @oschirg @Die...,0.0,0.0,0


### Function: Engineer Weighted Sentiment Feature

In [152]:
def sample_feature_engineering(df_sample):
    '''
    Generates the following features: "Weighted Sentiment", "Share of positive tweets", "Share of negative tweets".
    '''
    # Generate "Weighted Sentiment"
    df_sample["likes_count"] = df_sample["likes_count"]+10
    df_sample["retweets_count"] = df_sample["retweets_count"]+10
    df_sample["weighted_sentiment"] = np.log10(df_sample["likes_count"]) * np.log10(df_sample["retweets_count"]) * df_sample["sentiment"]

    # Generate "Share of positive tweets"
    df_sample["share_of_positive_tweets"] = df_sample["sentiment"]    
    dict_only_positive = {-1: 0, 0: 0, 1: 1}
    df_sample["share_of_positive_tweets"].replace(dict_only_positive, inplace=True)
    
    # Generate "Share of negative tweets"
    df_sample["share_of_negative_tweets"] = df_sample["sentiment"]
    dict_only_negative = {-1: 1, 0: 0, 1: 0}
    df_sample["share_of_negative_tweets"].replace(dict_only_negative, inplace=True)
    
    # Return DF
    return df_sample

In [153]:
df_sample_features = sample_feature_engineering(df_sample)
df_sample_features

,party,created_at,date,tweet,retweets_count,likes_count,sentiment,weighted_sentiment,share_of_positive_tweets,share_of_negative_tweets
0,GRUENE,2021-08-22 10:02:10,2021-08-22,@michfleischer @zeitonline @rezomusik @HeikoMa...,10.0,36.0,-1,-1.556303,0,1
1,FDP,2021-08-16 17:42:06,2021-08-16,@dan_uschi @mick_noir @fdp @c_lindner Guck dir...,10.0,10.0,-1,-1.000000,0,1
2,AFD,2021-08-23 06:50:05,2021-08-23,@Alice_Weidel Es kommen doch keine Afghanen ra...,10.0,10.0,-1,-1.000000,0,1
3,CDU,2021-08-23 11:35:31,2021-08-23,@ulfposh @welt @ArminLaschet Der liebe Armin s...,10.0,13.0,0,0.000000,0,0
4,AFD,2021-08-18 13:21:13,2021-08-18,@InneKomm @AfD_Muenster Eintagsfliegen geben d...,10.0,10.0,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...
995,CDU,2021-08-20 21:16:05,2021-08-20,@mija_roe @SoniaMarima @ArminLaschet @JoachimS...,10.0,10.0,-1,-1.000000,0,1
996,GRUENE,2021-08-23 21:20:41,2021-08-23,@JulianFSchmidt @WEEDY42O @Volker_Pudt @warrin...,10.0,10.0,0,0.000000,0,0
997,CDU,2021-08-17 17:14:28,2021-08-17,@derya_tn @WerteUnion @ArminLaschet Da weiß ma...,10.0,23.0,-1,-1.361728,0,1
998,GRUENE,2021-08-20 15:59:56,2021-08-20,@KPKuhn @Arishna12 @VeroWendland @oschirg @Die...,10.0,10.0,0,0.000000,0,0


## Poll Sample

In [51]:
data_poll = pd.read_csv('/Users/finnzurmuehlen/Downloads/pools_data_2021.csv')
data_poll.tail()

,Date,Surveyed_Persons,SPD,CDU/CSU,Grüne,FDP,AfD,Linke,other
114,2021-08-18,4874,20.67,23.00,18.83,11.67,10.67,6.67,8.50
115,2021-08-19,1385,20.50,24.17,17.83,12.00,11.00,7.17,7.33
116,2021-08-20,1179,11.00,0.00,7.00,6.00,21.00,10.00,45.00
117,2021-08-21,1265,16.50,11.00,12.00,9.50,16.50,8.50,26.00
118,2021-08-22,1352,22.00,22.00,17.00,13.00,12.00,7.00,7.00


## Pandas Engineer Features Table

### Cleaning Function

In [61]:
def cleaning(party, csv_path):
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=[#'Unnamed: 0',
                              "timezone",
                              "username", 
                              "name", 
                              "place", 
                              "urls", 
                              "photos", 
                              "cashtags", 
                              "retweet", 
                              "quote_url",
                              "video",
                              "thumbnail", 
                              "near", 
                              "source", 
                              "user_rt_id", 
                              "user_rt", 
                              "retweet_id", 
                              "reply_to",  
                              "retweet_date", 
                              "translate", 
                              "trans_src", 
                              "trans_dest",
                              "link",
                              "geo",
                              "time",
                              "mentions"
                                 ])
    df_party_name = df_party_name[(df_party_name.created_at.str.len() == 23) | (df_party_name.created_at.str.len() == 24)]
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"])
    df_party_name["date"] = pd.to_datetime(df_party_name["date"])
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name = df_party_name[(df_party_name.language == "de")]
    df_party_name = df_party_name.drop_duplicates()
    df_party_name.insert(loc=0, column="party", value=party)
    df_party_name = df_party_name.reset_index(drop=True)

    return df_party_name

In [58]:
df_cdu = cleaning("CDU",'/Users/finnzurmuehlen/Downloads/CDU_2308_v1 (1).csv')

/var/folders/dh/flkqfhts3lz7z3m3965czbf40000gn/T/ipykernel_57770/3187127069.py:1: DtypeWarning: Columns (10,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_cdu = cleaning("CDU",'/Users/finnzurmuehlen/Downloads/CDU_2308_v1 (1).csv')


In [59]:
df_afd = cleaning("AFD",'/Users/finnzurmuehlen/code/NicolasBuehringer/project_delphi/raw_data/afd_2308_v1.csv')

/var/folders/dh/flkqfhts3lz7z3m3965czbf40000gn/T/ipykernel_57770/897255081.py:1: DtypeWarning: Columns (0,1,2,10,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_afd = cleaning("AFD",'/Users/finnzurmuehlen/code/NicolasBuehringer/project_delphi/raw_data/afd_2308_v1.csv')


In [60]:
df_fdp = cleaning("FDP", '/Users/finnzurmuehlen/code/NicolasBuehringer/project_delphi/raw_data/fdp_2308_v1.csv')

/var/folders/dh/flkqfhts3lz7z3m3965czbf40000gn/T/ipykernel_57770/3978747379.py:1: DtypeWarning: Columns (0,1,2,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_fdp = cleaning("FDP", '/Users/finnzurmuehlen/code/NicolasBuehringer/project_delphi/raw_data/fdp_2308_v1.csv')


In [62]:
df_gruene = cleaning("GRUENE", '/Users/finnzurmuehlen/Downloads/gruene.csv')

/var/folders/dh/flkqfhts3lz7z3m3965czbf40000gn/T/ipykernel_57770/1775248056.py:1: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df_gruene = cleaning("GRUENE", '/Users/finnzurmuehlen/Downloads/gruene.csv')


In [63]:
df_linke = cleaning("LINKE", '/Users/finnzurmuehlen/Downloads/linke.csv')

In [64]:
df_spd = cleaning("SPD", '/Users/finnzurmuehlen/Downloads/spd.csv')

### Concat Function

In [65]:
liste = [df_afd, df_cdu, df_fdp, df_gruene, df_linke, df_spd]

In [66]:
def concat(liste):
    df_all = pd.concat(liste)
    df_all = df_all.reset_index(drop=True)
    df_all["avg_len_of_tweet"] = df_all["tweet"].str.len()
    return df_all

In [67]:
df_all = concat(liste)

### Feature Engineering

In [68]:
def pandas_feature_creation(df):
    
    # Create: "Avg. len per tweet of each party"
    df_temp = df.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean"
    }) 
    
    # Create: share of tweets that a party has in comparison to all tweets on a given day 
    df_temp_2 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "tweet": "count"}).groupby(level=0).apply(lambda x: x/x.sum())
    
    # Create: Share of tweets that come from a unique user for each party on a given day
    df_temp_3 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "user_id": "nunique",
    "tweet": "count"})
    df_temp_3["share_unique_users"] = df_temp_3["user_id"] / df_temp_3["tweet"]
    df_temp_3 = df_temp_3["share_unique_users"]
    
     # Join the different temporary DFs into a final DataFrame
    df_final = df_temp.join(df_temp_2).join(df_temp_3)
    df_final = df_final.rename(columns={'tweet': "share_of_tweets"})
    
    return df_final
    

### Table with pandas features

In [71]:
data_features = pandas_feature_creation(df_all)
data_features

replies_count  retweets_count  likes_count  \
created_at party                                                
2021-08-15 AFD             278.0           471.0       1682.0   
           CDU           12520.0         24679.0     249082.0   
           FDP            4549.0          3476.0      38675.0   
           GRUENE         1702.0          3116.0      20471.0   
           LINKE           332.0           576.0       3627.0   
           SPD            1148.0          1996.0      14589.0   
2021-08-16 AFD            7821.0         16054.0      94816.0   
           CDU           30584.0         40776.0     351880.0   
           FDP            3179.0          3012.0      29618.0   
           GRUENE         5554.0          8736.0      59198.0   
           LINKE          1201.0          1413.0       9016.0   
           SPD            3566.0          8864.0      54360.0   
2021-08-17 AFD            7224.0         16655.0      91083.0   
           CDU           11423.0         11373.0     104674.0   
           FDP            1489.0           958.0       8182.0   
           GRUENE         3801.0          4439.0      41800.0   
           LINKE          1360.0          1709.0       8872.0   
           SPD            3949.0          5370.0      32040.0   
2021-08-18 AFD            6488.0         12879.0      73847.0   
           CDU           12106.0         17459.0     116347.0   
           FDP            1510.0          1925.0      13444.0   
           GRUENE         4254.0          4293.0      27632.0   
           LINKE          1359.0          1073.0       7685.0   
           SPD            5339.0          9002.0      55124.0   
2021-08-19 AFD           17887.0         31569.0     191927.0   
           CDU            9004.0         11633.0      93015.0   
           FDP            1727.0          1222.0      11124.0   
           GRUENE         6249.0          7946.0      61616.0   
           LINKE          1099.0          1633.0       8407.0   
           SPD            3840.0          5417.0      38124.0   
2021-08-20 AFD           12249.0         17768.0     101640.0   
           CDU           15183.0         19001.0     152565.0   
           FDP            7265.0          3711.0      59321.0   
           GRUENE         8632.0          9330.0      69674.0   
           LINKE           813.0          1305.0       6908.0   
           SPD            4592.0          5427.0      51127.0   
2021-08-21 AFD            5801.0         12270.0      67105.0   
           CDU           36219.0         34297.0     291497.0   
           FDP            1329.0          1252.0      10686.0   
           GRUENE         4038.0          3282.0      26306.0   
           LINKE           740.0           651.0       4053.0   
           SPD            3560.0          5346.0      33256.0   
2021-08-22 AFD            7160.0         15148.0      75008.0   
           CDU           23978.0         38584.0     307038.0   
           FDP            1979.0          1700.0      15553.0   
           GRUENE         9852.0         10488.0      80533.0   
           LINKE          1470.0          1443.0       7590.0   
           SPD            6824.0          6573.0      48056.0   
2021-08-23 AFD            1779.0          4084.0      14931.0   
           CDU           12068.0         16166.0     129789.0   
           FDP            1783.0          1957.0      13470.0   
           GRUENE         5491.0          6017.0      37603.0   
           LINKE          1551.0          1344.0       9400.0   
           SPD            5335.0          4363.0      34300.0   
2021-08-24 AFD              14.0             6.0         34.0   
           FDP               6.0             3.0         19.0   
           GRUENE           15.0             5.0         23.0   
           LINKE             5.0             1.0          5.0   
           SPD              31.0            18.0         73.0   

                   avg_len_of_tweet  share_of_tweets 

## RNN Model Table